In [1]:
import sys
currentDir=%pwd
sys.path.append(currentDir)
import gipsy

In [2]:
from bioservices import UniProt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import matplotlib.ticker as ticker
import ipc
from IPython.display import display, Image, HTML
 
u = UniProt()

In [3]:
x = 12
y = 10


pH=7.4
#Insert uniprot Taxonomy ID
taxonomyID=83333
taxonomy_name="Escherichia coli \(strain K12"

csv_folder="Raw"
cut_folder="Cuts"
dbfile="Ecoli_Hres_db.csv"

tryp_csv_folder="Tryp_Raw"
tryp_cut_folder="Tryp_Cut"
tryp_dbfile="Tryp_Ecoli_Hres_db.csv"


In [4]:
%time df_gipsy=gipsy.CSVconverter(cut_folder=cut_folder,input_folder=csv_folder,dbfile=dbfile,x=x,y=y,taxonomy=taxonomy_name,sig_seq=0)
%time df_tryp=gipsy.TrypsinCSVconverter(cut_folder=tryp_cut_folder,input_folder=tryp_csv_folder,dbfile=tryp_dbfile,x=x,y=y)

CPU times: user 4min 32s, sys: 2.29 s, total: 4min 34s
Wall time: 6min 24s
CPU times: user 12.9 s, sys: 104 ms, total: 13.1 s
Wall time: 13.2 s


In [4]:
df_gipsy=pd.read_csv(dbfile)

In [6]:
df_tryp

,prot_acc,prot_desc,A1,A2,A3,A4,A5,A6,A7,A8,...,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10
0,P00761,TRYP_PIG Trypsin OS=Sus scrofa PE=1 SV=1,0.5,0.49,0.49,0.48,0.78,0.47,0.82,0.48,...,0.51,0.49,0.47,0.47,0.83,0.79,0.46,0.77,0.47,0.47


In [5]:
sns.set_style("darkgrid")
sns.set(font_scale=1.3)
ecoli_entry=u.search("taxonomy:%s AND reviewed:yes"%taxonomyID,frmt='list').split('\n')
total_protein_taxonomy=len(ecoli_entry)
total_protein_gipsy=len(df_gipsy)


hist_x=["Total:","proteins:GIPSY"]
hist_y=[total_protein_taxonomy,total_protein_gipsy]
plot_data_total=pd.DataFrame({'X':hist_x,'Y':hist_y})

fig,ax1 = plt.subplots(figsize=(9,6))

p_left_bg=sns.barplot(plot_data_total['X'],
                      y=plot_data_total['Y'],
                      palette="Set1",
                      ax=ax1,
                     )
    
for p in p_left_bg.patches:
    px=p.get_bbox().get_points()[:,0]
    py=p.get_bbox().get_points()[1,1]
    p_left_bg.annotate('{:.1f}%\n(n={:.0f})'.format(100.*py/total_protein_taxonomy,py), (px.mean(), py+50), 
            ha='center', va='bottom',size=19)
    
p_left_bg.set_ylim(0,total_protein_taxonomy*1.25)

Tax=mpatches.Patch(color=sns.color_palette("Set1")[0], label="Taxonomy")
Gip=mpatches.Patch(color=sns.color_palette("Set1")[1], label="GIPSY")
Nsmear=mpatches.Patch(color=sns.color_palette("hls",2)[0], label="No Smear")
Smear=mpatches.Patch(color=sns.color_palette("hls",2)[1], label="Smear")

ax1.axhline(y=0,color='grey') 

ax1.legend(handles=[Tax,Gip],loc=0,fontsize='x-large',frameon=True,facecolor='white')

ax1.axes.get_xaxis().set_visible(False)

p_left_bg.set_ylabel("Protein Count",size=15)

ax1.set_title("Amount of Tanoxomy Found",size=25,y=1.08)

plt.tight_layout()
plt.savefig("%s_taxo.png"%dbfile.split('_db')[0],dpi=500)
plt.close(fig)

In [11]:
sns.set(style="darkgrid")
sns.set(font_scale=1.3)

fig=plt.subplots(figsize=(15,10))

ncount=len(df_gipsy)

ax2=plt.subplot(221)
ax3=plt.subplot(212)
ax1=plt.subplot(222)


p_left=sns.countplot(df_gipsy["Category"],
                      ax=ax2,
                      order=["found","useful",'overhalf'],
                      palette=sns.color_palette("husl",3))


p_right=sns.countplot(df_gipsy["num_pixels"],hue=df_gipsy["Category"],
                      hue_order=["found","useful",'overhalf'],
                      ax=ax3,
                      palette=sns.color_palette("husl",3))

p_first=sns.countplot(df_gipsy["Smear"],
                      ax=ax1,
                      palette=sns.color_palette("hls",2))


for p in p_left.patches:
    px=p.get_bbox().get_points()[:,0]
    py=p.get_bbox().get_points()[1,1]
    p_left.annotate('{:.1f}%\n(n={:.0f})'.format((100.*py)/ncount,py), (px.mean(), py+5), 
            ha='center', va='bottom',size=20)
    
for p in p_first.patches:
    px=p.get_bbox().get_points()[:,0]
    py=p.get_bbox().get_points()[1,1]
    p_first.annotate('{:.1f}%\n(n={:d})'.format(100.*py/ncount,int(py)), (px.mean(), py*0.5), 
            ha='center', va='center',size=19,color='white')

usef=mpatches.Patch(color=sns.color_palette("husl",3)[1], label="Useful")
fnd=mpatches.Patch(color=sns.color_palette("husl",3)[0], label="Found")
ofh=mpatches.Patch(color=sns.color_palette("husl",3)[2], label="Overhalf")
Nsmear=mpatches.Patch(color=sns.color_palette("hls",2)[0], label="No Smear")
Smear=mpatches.Patch(color=sns.color_palette("hls",2)[1], label="Smear")

ax2.axhline(y=0,color='grey') 
ax3.axhline(y=0,color='grey')
ax1.axhline(y=0,color='grey')

ax2.legend(handles=[fnd,usef,ofh],loc=0,fontsize=25,frameon=True,facecolor='white')
ax3.legend(handles=[fnd,usef,ofh],loc=9,ncol=3,fontsize=25,frameon=True,facecolor='white')
ax1.legend(handles=[Nsmear,Smear],loc=0,fontsize=25,frameon=True,facecolor='white')


ax2.axes.get_xaxis().set_visible(False)
ax1.axes.get_xaxis().set_visible(False)


p_left.set_ylim(0,ncount*1.20)
p_right.set_ylabel("Protein Count",size=20)
p_left.set_ylabel("Protein Count",size=20)
p_first.set_ylabel("Protein Count",size=20)
p_right.set_xlabel("Number of active Pixels",size=20)

ax1.yaxis.set_tick_params(labelsize=16)
ax2.yaxis.set_tick_params(labelsize=16)
ax3.yaxis.set_tick_params(labelsize=16)
plt.setp(ax3.get_xticklabels(), fontsize=10, rotation=60)

ax2.set_title("Categories - Initial",size=36,y=1.05)
ax3.set_title("Distribution of active pixels after",size=36,y=1.05)
ax1.set_title("Smear distribution - Initial",size=36,y=1.05)
plt.tight_layout()
plt.savefig("%s_initial_category.png"%dbfile.split('_db')[0],dpi=500)
plt.close()


In [6]:
sns.set_style("darkgrid")
sns.set(font_scale=1.3)
df_prot_pI=df_gipsy[["prot_acc","prot_mass","pI"]]
df_prot_pI=df_prot_pI.apply(pd.to_numeric, errors='ignore')

ent=u.search("taxonomy:%s AND reviewed:yes"%taxonomyID,frmt='tab',columns='entry name,protein names,mass,sequence').split('\n')
df_taxo=pd.DataFrame([sub.split("\t") for sub in ent],columns=['prot_acc','prot_desc','prot_mass','sequence'])

df_taxo=df_taxo[1:]
df_taxo['pI']=0
df_taxo.dropna(inplace=True)
for index,row in df_taxo.iterrows():
    seq=row["sequence"]
    p_i=ipc.predict_isoelectric_point(seq)
    df_taxo.loc[index,"pI"]=p_i

fig, (ax1,ax2) = plt.subplots(nrows=1,ncols=2,figsize=(15,6))    

current_palette = sns.color_palette()
taxo_legend=mpatches.Patch(color=current_palette[0], label="Taxonomy")
gipsy_legend=mpatches.Patch(color=current_palette[1], label="GIPSY")
Gel_pH=mpatches.Patch(color='red',label="pH of gel")
gipsy_prot_mass_min=(df_prot_pI['prot_mass'].min())/1000
gipsy_prot_mass_max=(df_prot_pI['prot_mass'].max())/1000
prot_mm=mpatches.Patch(color='grey',label='Min-Max')
prot_mmnum=mpatches.Patch(color='none',label='%s-%s kDa'%(int(round(gipsy_prot_mass_min)),
                                                          int(round(gipsy_prot_mass_max))))

df_taxo["prot_mass"] =df_taxo["prot_mass"].str.replace(",", "").astype(np.int)
df_taxo["prot_mass"]=pd.to_numeric(df_taxo["prot_mass"])

ax1_twin=ax1.twinx()
ax2_twin=ax2.twinx()
prot_taxo=sns.distplot(df_taxo["prot_mass"]/1000,color=current_palette[0],kde=False,ax=ax1_twin)
prot_gipsy=sns.distplot(df_prot_pI["prot_mass"]/1000,color=current_palette[1],kde=False,ax=ax1)

pI_taxo=sns.distplot(df_taxo["pI"],color=current_palette[0],kde=False,ax=ax2_twin)
pI_gipsy=sns.distplot(df_prot_pI["pI"],color=current_palette[1],kde=False,ax=ax2)

ax1.yaxis.grid(False)
ax1_twin.yaxis.grid(False)
ax2.yaxis.grid(False)
ax2_twin.yaxis.grid(False)

ax1.legend(handles=[taxo_legend,gipsy_legend,prot_mm,prot_mmnum],loc=0,fontsize=20,frameon=True,facecolor='white')
ax2.legend(handles=[taxo_legend,gipsy_legend,Gel_pH],loc=0,fontsize=20,frameon=True,facecolor='white')

ax1.set_ylabel("GIPSY Protein Count",size=15,color=current_palette[1])
ax2.set_ylabel("GIPSY Protein Count",size=15,color=current_palette[1])
ax1_twin.set_ylabel("Taxonomy Protein Count",size=15,color=current_palette[0])
ax2_twin.set_ylabel("Taxonomy Protein Count",size=15,color=current_palette[0])

ax1.set_xlabel("Protein masses (kDa)",size=15)
ax2.set_xlabel("pH",size=15)

ax1.set_xlim(0,max(df_taxo["prot_mass"]/1000))
ax1.set_xticks(np.arange(0,max(df_taxo["prot_mass"]/1000),25))

ax2.set_xticks(np.arange(0,max(df_taxo["pI"]),1))
ax2.set_xlim(min(df_taxo["pI"]),max(df_taxo["pI"]))

ax1.set_title("Distubution of protein masses\ntaxonomy and GIPSY",size=25,y=1.05)
ax2.set_title("Distubution of isoelectric point\ntaxonomy and GIPSY",size=25,y=1.05)

ax1.axhline(y=0,color='grey')
ax1.axvline(x=gipsy_prot_mass_min,color='grey')
ax1.axvline(x=gipsy_prot_mass_max,color='grey')
ax2.axhline(y=0,color='grey')
ax2.axvline(x=pH,color='red')

ax1.tick_params(axis='y',colors=current_palette[1])
ax1_twin.tick_params(axis='y',colors=current_palette[0])

ax2.tick_params(axis='y',colors=current_palette[1])
ax2_twin.tick_params(axis='y',colors=current_palette[0])

plt.tight_layout()
plt.savefig("%s_mass_pI.png"%dbfile.split('_db')[0],dpi=500)
plt.close()
